This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'some key'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import json
import requests

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
afx_x_single_data = requests.get(f'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2020-01-01&end_date=2020-01-01&api_key={API_KEY}')

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(afx_x_single_data.json())

{'quandl_error': {'code': 'QEAx01', 'message': 'We could not recognize your API key: some key. Please check your API key and try again. You can find your API key under your account settings.'}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
# Question 1 - Retrieving 2017 data
afx_x_2017 = requests.get(f'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key={API_KEY}')

In [6]:
# 2017 Dataset
afx_year = afx_x_2017.json()

In [7]:
# Question 2 - Convert JSON to Dictionary
afx_year = json.dumps(afx_year) # Convert json to string
afx_year = json.loads(afx_year) # Convert stringified json to dict

In [8]:
# Organizing data
afx_dataset = afx_year['dataset_data']
afx_column_names = afx_dataset['column_names']
afx_data = afx_dataset['data']
afx_dict = {}

# Create dict with column names as keys and empty list as value
for col_name in afx_column_names:
    afx_dict[col_name] = []

# Adding additional key - daily change (high - Low) and closing change (second day - first day)
afx_dict['Daily Change'] = []
afx_dict['Closing Change'] = []

# Appending data to the list based on category
for data in afx_data:
    
    # Removing None type
    if (data[1] != None):
        afx_dict['Open'].append(data[1])
    
    # Finding daily change
    if (data[2:3] != None) :
        afx_dict['Daily Change'].append(data[2] - data[3])
        
    afx_dict['Date'].append(data[0])
    afx_dict['High'].append(data[2])
    afx_dict['Low'].append(data[3])
    afx_dict['Close'].append(data[4])
    afx_dict['Change'].append(data[5])
    afx_dict['Traded Volume'].append(data[6])
    afx_dict['Turnover'].append(data[7])
    afx_dict['Last Price of the Day'].append(data[8])
    afx_dict['Daily Traded Units'].append(data[9])
    afx_dict['Daily Turnover'].append(data[10])


KeyError: 'dataset_data'

In [ ]:
# Qustion 3 - Find lowest and highest opening stock
open_data = afx_dict['Open']
highest_open_stock = max(open_data)
lowest_open_stock = min(open_data)

print(f'The lowest opening stock price: ${lowest_open_stock}')
print(f'The highest opening stock price: ${highest_open_stock}')

In [ ]:
# Question 4 - Largest daily change stock price
highest_daily_change = round(max(afx_dict['Daily Change']), 2) # Round to two decimals

print(f'The highest daily change in stock price: ${highest_daily_change}')

In [ ]:
# Question 5 - Largest change between two days based on closing price
close_prices = afx_dict['Close']
largest_change = 0

for idx, value in enumerate(close_prices):
    
    # Find difference between current index and previous index
    if (idx > 0):
        price_diff = round(abs(value - close_prices[idx - 1]), 2) # Round by two decimals
    
    # Define largest change in price
    if price_diff > largest_change:
        largest_change = price_diff
        
print(f'The highest change in stock closing price between two days is ${largest_change}')

In [ ]:
# Question 6 - Find average daily trading volume
traded_volume = afx_dict['Traded Volume']
avg_traded_volume = round(sum(traded_volume) / len(traded_volume))

print(f'Average daily trading volume: {avg_traded_volume}')

In [ ]:
# Question 7 - Find median trading volume
traded_volume.sort()
length = len(traded_volume)
mid = int(length / 2)

# Odd total number
if length % 2 == 1:
    vol_median = traded_volume[mid]
else:
    first_mid = traded_volume[mid - 1]
    second_mid = traded_volume[mid]
    vol_median = (first_mid + second_mid) / 2
    
print(f'Median daily trading volume: {vol_median}')